# Ethnicity,Age,Gender Bias in Stop and Search Practices

In [1]:
import pandas as pd
import sqlite3
from scipy.stats import kruskal
import statsmodels.formula.api as smf

# Load the stop-and-search data
conn = sqlite3.connect('data/cleaned_police_data-2.db')
stop_and_search_df = pd.read_sql_query("SELECT * from stop_and_search", conn)
conn.close()

# Function for preprocessing officer defined ethnicity
def group_ethnicity(row):
    ethnicity = row['officer_defined_ethnicity'].lower()
    if 'white' in ethnicity:
        return 'White'
    elif 'black' in ethnicity:
        return 'Black'
    elif 'asian' in ethnicity or 'chinese' in ethnicity:
        return 'Asian'
    elif 'mixed' in ethnicity or 'other' in ethnicity or 'not stated' in ethnicity:
        return 'Mixed/Other'
    else:
        return 'Unknown'

# Data cleaning steps
stop_and_search_df = stop_and_search_df[stop_and_search_df['borough'] != 'city of london']
stop_and_search_df.drop(['policing_operation', 'outcome_linked_to_object_of_search', 'removal_of_more_than_just_outer_clothing'], axis=1, inplace=True)
stop_and_search_df.dropna(inplace=True)
stop_and_search_df['month'] = pd.to_datetime(stop_and_search_df['month'])
stop_and_search_df['Year'] = stop_and_search_df['month'].dt.year
stop_and_search_df['Month'] = stop_and_search_df['month'].dt.month
stop_and_search_df = stop_and_search_df.drop('month', axis=1)
stop_and_search_df['borough'] = stop_and_search_df['borough'].str.lower()
stop_and_search_df['grouped_ethnicity'] = stop_and_search_df.apply(group_ethnicity, axis=1)

# Load the population data from the Excel file
file_path = 'data/ethnic-groups-by-borough.xls'
population_df = pd.read_excel(file_path, sheet_name=None, engine='xlrd')

# Function to restructure each sheet
def restructure_sheet(df, year):
    df = df.drop(0)
    df = df.drop(columns=['Code'])
    df.columns = ['borough', 'White', 'Asian', 'Black', 'Mixed/Other', 'Total', 'Unnamed: 7', '95% Confidence Interval', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']
    df = df.drop(columns=['Unnamed: 7', '95% Confidence Interval', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'])
    df['Year'] = year
    return df

# List to hold restructured DataFrames
restructured_dfs = []
for sheet_name in population_df.keys():
    if sheet_name != 'Metadata':
        df = population_df[sheet_name]
        restructured_df = restructure_sheet(df, sheet_name)
        restructured_dfs.append(restructured_df)

# Combine all restructured DataFrames into one
combined_df = pd.concat(restructured_dfs, ignore_index=True)
combined_df['borough'] = combined_df['borough'].str.lower()

# Define the set of boroughs from stop and search
stop_and_search_boroughs = { 'barking and dagenham', 'barnet', 'bexley', 'brent', 'bromley', 'camden', 'croydon', 'ealing', 'enfield', 'greenwich', 'hackney', 'hammersmith and fulham', 'haringey', 'harrow', 'havering', 'hillingdon', 'hounslow', 'islington', 'kensington and chelsea', 'kingston upon thames', 'lambeth', 'lewisham', 'merton', 'newham', 'redbridge', 'richmond upon thames', 'southwark', 'sutton', 'tower hamlets', 'waltham forest', 'wandsworth', 'westminster' }
filtered_df = combined_df[combined_df['borough'].isin(stop_and_search_boroughs)]

# Convert relevant columns to numeric and identify problematic rows
problematic_rows = {}
for col in ['White', 'Asian', 'Black', 'Mixed/Other', 'Total']:
    filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')
    problematic_rows[col] = filtered_df[filtered_df[col].isnull()]

for col, rows in problematic_rows.items():
    if not rows.empty:
        print(f"Problematic rows for column '{col}':")
        print(rows)

filtered_df['Black'] = filtered_df.groupby('borough')['Black'].transform(lambda x: x.fillna(x.median()))

filtered_df['Year'] = filtered_df['Year'].astype(int)
stop_and_search_df['Year'] = stop_and_search_df['Year'].astype(int)

# Rename columns in filtered_df to avoid conflicts
filtered_df.rename(columns={ 'White': 'White_population', 'Asian': 'Asian_population', 'Black': 'Black_population', 'Mixed/Other': 'Mixed_Other_population' }, inplace=True)

# Calculate total stop-and-search counts by ethnicity and borough
search_counts = stop_and_search_df.groupby(['borough', 'Year', 'Month', 'grouped_ethnicity']).size().unstack(fill_value=0).reset_index()

# Merge with population data
merged_df = pd.merge(search_counts, filtered_df, on=['borough', 'Year'], how='inner')

# Calculate stop-and-search percentages for each ethnicity
merged_df['White_percentage'] = (merged_df['White'] / merged_df['White_population']) * 100
merged_df['Asian_percentage'] = (merged_df['Asian'] / merged_df['Asian_population']) * 100
merged_df['Black_percentage'] = (merged_df['Black'] / merged_df['Black_population']) * 100
merged_df['Mixed_Other_percentage'] = (merged_df['Mixed/Other'] / merged_df['Mixed_Other_population']) * 100

stop_and_search_df.drop(['part_of_a_policing_operation', 'latitude', 'longitude', 'self_defined_ethnicity',
'officer_defined_ethnicity','legislation','object_of_search','outcome'], axis=1, inplace=True)

# Create dummy variables for age_range and gender
stop_and_search_df = pd.get_dummies(stop_and_search_df, columns=['age_range','gender'])

# Calculate the total number of stops per borough, year, and month
total_stops = stop_and_search_df.groupby(['borough','Year','Month']).size().reset_index(name='total_stops')

# Calculate the number of stops per age group and gender
age_gender_counts = stop_and_search_df.groupby(['borough','Year','Month','grouped_ethnicity']).sum().reset_index()
age_gender_counts.drop(['type'],axis=1,inplace=True)

# Merge the total stops with age_gender_counts
merged_counts=pd.merge(age_gender_counts,total_stops,on=['borough','Year','Month'])

# Calculate proportions for age groups and gender
for column in age_gender_counts.columns[4:]:
    merged_counts[column]=merged_counts[column]/merged_counts['total_stops']

# Select only the proportion columns and identifiers
age_gender_proportions=merged_counts[['borough','Year','Month','grouped_ethnicity']+list(age_gender_counts.columns[4:])]

# Pivot the DataFrame to align age and gender proportions with ethnicity percentages
pivot_df = merged_df.melt(
    id_vars=['borough', 'Year', 'Month'],
    value_vars=['White_percentage',
                'Asian_percentage',
                'Black_percentage',
                'Mixed_Other_percentage'],
    var_name='ethnicity',
    value_name='percentage'
)

# Map ethnicity names to grouped_ethnicity for merging
ethnicity_map = {
    'White_percentage': 'White',
    'Asian_percentage': 'Asian',
    'Black_percentage': 'Black',
    'Mixed_Other_percentage': 'Mixed/Other'
}
pivot_df['grouped_ethnicity'] = pivot_df['ethnicity'].map(ethnicity_map)

# Merge the pivoted DataFrame with age_gender_proportions
pivot_df = pd.merge(pivot_df, age_gender_proportions, on=['borough', 'Year', 'Month', 'grouped_ethnicity'], how='left')

# Set the reference category for `ethnicity` to `White_percentage`
pivot_df['ethnicity'] = pivot_df['ethnicity'].astype('category')
pivot_df['ethnicity'] = pivot_df['ethnicity'].cat.reorder_categories(
    ['White_percentage', 
     'Asian_percentage',
     'Black_percentage',
     'Mixed_Other_percentage'], ordered=True
)


# Update column names accordingly
pivot_df.columns = [col.replace('-', '_') for col in pivot_df.columns]
pivot_df.rename(columns={'age_range_over 34': 
                         'age_range_over_34',
                         'age_range_under 10': 
                         'age_range_under_10'}, inplace=True)


pivot_df_clean = pivot_df.dropna()

pivot_df_clean.drop(columns=['grouped_ethnicity','age_range_under_10','gender_other'])


/var/folders/j3/c6lj_2051kdcm9d4lrwsyf0r0000gn/T/ipykernel_87769/2423060629.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')
/var/folders/j3/c6lj_2051kdcm9d4lrwsyf0r0000gn/T/ipykernel_87769/2423060629.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Black'] = filtered_df.groupby('borough')['Black'].transform(lambda x: x.fillna(x.median()))
/var/folders/j3/c6lj_2051kdcm9d4lrwsyf0r0000gn/T/ipykernel_87769/2423060629.py:78

Problematic rows for column 'Black':
                  borough   White  Asian  Black Mixed/Other   Total  Year
27   richmond upon thames  163000  14000    NaN       18000  200000  2020
125  richmond upon thames  170000  11000    NaN       14000  196000  2018
370  richmond upon thames  168000  10000    NaN       10000  189000  2013
419  richmond upon thames  164000  11000    NaN       10000  186000  2012


,borough,Year,Month,ethnicity,percentage,age_range_10_17,age_range_18_24,age_range_25_34,age_range_over_34,gender_female,gender_male
0,barking and dagenham,2016,6,White_percentage,0.100943,0.055814,0.265116,0.102326,0.074419,0.032558,0.465116
1,barking and dagenham,2016,7,White_percentage,0.086792,0.039604,0.237624,0.094059,0.084158,0.049505,0.405941
2,barking and dagenham,2016,8,White_percentage,0.102830,0.105820,0.285714,0.142857,0.042328,0.042328,0.534392
3,barking and dagenham,2016,9,White_percentage,0.110377,0.100917,0.233945,0.119266,0.082569,0.041284,0.495413
4,barking and dagenham,2016,10,White_percentage,0.196226,0.126050,0.221289,0.156863,0.078431,0.044818,0.537815
...,...,...,...,...,...,...,...,...,...,...,...
6851,westminster,2020,8,Mixed_Other_percentage,0.527273,0.028740,0.105664,0.047337,0.014370,0.007608,0.188504
6852,westminster,2020,9,Mixed_Other_percentage,0.381818,0.033133,0.070281,0.044177,0.021084,0.004016,0.164659
6853,westminster,2020,10,Mixed_Other_percentage,0.570455,0.051793,0.082869,0.043825,0.021514,0.006375,0.193625
6854,westminster,2020,11,Mixed_Other_percentage,0.552273,0.050682,0.115010,0.052632,0.018519,0.002924,0.233918


In [2]:
# Fit the mixed-effects model with age and gender proportions as covariates
model = smf.mixedlm(
    "percentage ~ ethnicity + age_range_18_24 + age_range_25_34 + age_range_over_34 + gender_female + gender_male", 
    pivot_df_clean, 
    groups=pivot_df_clean["borough"], 
    re_formula="~Month"
)
result = model.fit()

# Print the summary of the model
print(result.summary())

                    Mixed Linear Model Regression Results
Model:                    MixedLM        Dependent Variable:        percentage
No. Observations:         6710           Method:                    REML      
No. Groups:               32             Scale:                     0.0704    
Min. group size:          190            Log-Likelihood:            -698.5695 
Max. group size:          216            Converged:                 Yes       
Mean group size:          209.7                                               
------------------------------------------------------------------------------
                                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------
Intercept                            0.016    0.033  0.480 0.631 -0.049  0.081
ethnicity[T.Asian_percentage]        0.091    0.015  6.121 0.000  0.062  0.120
ethnicity[T.Black_percentage]        0.441    0.013 34.992 0.000  0.417  

/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


In [3]:
# Fit the mixed-effects model with interaction terms directly in the formula
model_interaction = smf.mixedlm(
    "percentage ~ C(ethnicity) * age_range_18_24 + C(ethnicity) * gender_female", 
    pivot_df_clean, 
    groups=pivot_df_clean["borough"], 
    re_formula="~Month"
)
result_interaction = model_interaction.fit()

# Print the summary of the model
print(result_interaction.summary())

                              Mixed Linear Model Regression Results
Model:                           MixedLM              Dependent Variable:              percentage
No. Observations:                6710                 Method:                          REML      
No. Groups:                      32                   Scale:                           0.0699    
Min. group size:                 190                  Log-Likelihood:                  -666.4506 
Max. group size:                 216                  Converged:                       Yes       
Mean group size:                 209.7                                                           
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                                         

/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


# Impact of Age_Range on S&S outcomes

In [4]:
import pandas as pd
import sqlite3

# Load the stop-and-search data
conn = sqlite3.connect('data/cleaned_police_data-2.db')
stop_and_search_df = pd.read_sql_query("SELECT * from stop_and_search", conn)
conn.close()

# Data cleaning steps
stop_and_search_df = stop_and_search_df[stop_and_search_df['borough'] != 'city of london']
stop_and_search_df.drop(['policing_operation', 'outcome_linked_to_object_of_search', 'removal_of_more_than_just_outer_clothing'], axis=1, inplace=True)
stop_and_search_df.dropna(inplace=True)
stop_and_search_df['month'] = pd.to_datetime(stop_and_search_df['month'])
stop_and_search_df['Year'] = stop_and_search_df['month'].dt.year
stop_and_search_df['Month'] = stop_and_search_df['month'].dt.month
stop_and_search_df = stop_and_search_df.drop('month', axis=1)
stop_and_search_df['borough'] = stop_and_search_df['borough'].str.lower()

# Rename columns to avoid issues with mixed effects model
stop_and_search_df.columns = [col.replace(' ', '_').replace('-', '_').replace('(', '').replace(')', '').replace('/', '_') for col in stop_and_search_df.columns]

# Group by borough, Year, and Month and create dummy variables
grouped_df = stop_and_search_df.groupby(['borough', 'Year', 'Month']).sum().reset_index()

# Convert age_range and outcome columns to dummies
stop_and_search_df = pd.get_dummies(stop_and_search_df, columns=['age_range', 'outcome'], drop_first=True)

# Aggregate the stop_and_search data by borough, Year, Month, and outcome
grouped_df = stop_and_search_df.groupby(['borough', 'Year', 'Month']).sum().reset_index()

grouped_df.columns= [col.replace(' ', '_').replace('-', '_').replace('(', '').replace(')', '').replace('/', '_') for col in grouped_df.columns]

In [5]:
import statsmodels.formula.api as smf

grouped_df.columns= [col.replace(' ', '_').replace('-', '_').replace('(', '').replace(')', '').replace('/', '_') for col in grouped_df.columns]

# Define the outcome variables to analyze
outcomes = [
    'outcome_arrest',
    'outcome_nothing_found___no_further_action',
    'outcome_article_found___detailed_outcome_unavailable',
    'outcome_caution_simple_or_conditional',
    'outcome_community_resolution',
    'outcome_khat_or_cannabis_warning',
    'outcome_local_resolution',
    'outcome_offender_cautioned',
    'outcome_offender_given_drugs_possession_warning',
    'outcome_offender_given_penalty_notice',
    'outcome_penalty_notice_for_disorder',
    'outcome_summons___charged_by_post',
    'outcome_suspect_arrested',
    'outcome_suspect_summonsed_to_court'
]

# Fit Mixed Effects Logistic Regression model for each outcome
for outcome in outcomes:
    formula = f"{outcome} ~ age_range_under_10 + age_range_18_24 + age_range_25_34 + age_range_over_34"
    model = smf.mixedlm(formula, grouped_df, groups=grouped_df["borough"])
    result = model.fit()
    print(f"Results for {outcome}:")
    print(result.summary())
    print("\n")


Results for outcome_arrest:
            Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: outcome_arrest
No. Observations:  2898    Method:             REML          
No. Groups:        32      Scale:              596.3959      
Min. group size:   90      Log-Likelihood:     -13418.0660   
Max. group size:   92      Converged:          Yes           
Mean group size:   90.6                                      
-------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept          -4.155    1.854 -2.242 0.025 -7.788 -0.522
age_range_under_10 -0.200    1.416 -0.141 0.888 -2.976  2.576
age_range_18_24     0.023    0.013  1.720 0.085 -0.003  0.049
age_range_25_34     0.204    0.027  7.432 0.000  0.150  0.258
age_range_over_34   0.371    0.021 17.620 0.000  0.330  0.412
Group Var          75.881    0.894                    

/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


Results for outcome_nothing_found___no_further_action:
                         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: outcome_nothing_found___no_further_action
No. Observations: 2898    Method:             REML                                     
No. Groups:       32      Scale:              8428.5398                                
Min. group size:  90      Log-Likelihood:     -17265.0864                              
Max. group size:  92      Converged:          Yes                                      
Mean group size:  90.6                                                                 
-------------------------------------------------------------------------------------------
                          Coef.       Std.Err.        z        P>|z|     [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  97.426       10.741       9.070     0.000     76.37

/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


Results for outcome_article_found___detailed_outcome_unavailable:
                              Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: outcome_article_found___detailed_outcome_unavailable
No. Observations: 2898    Method:             REML                                                
No. Groups:       32      Scale:              0.0273                                              
Min. group size:  90      Log-Likelihood:     1068.0526                                           
Max. group size:  92      Converged:          Yes                                                 
Mean group size:  90.6                                                                            
---------------------------------------------------------------------------------------------------------
                            Coef.         Std.Err.          z           P>|z|        [0.025        0.975]
------------------------------------------------------------

...

/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


                      Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: outcome_penalty_notice_for_disorder
No. Observations: 2898    Method:             REML                               
No. Groups:       32      Scale:              54.0855                            
Min. group size:  90      Log-Likelihood:     -9965.1662                         
Max. group size:  92      Converged:          Yes                                
Mean group size:  90.6                                                           
-------------------------------------------------------------------------------------
                       Coef.      Std.Err.       z        P>|z|     [0.025     0.975]
-------------------------------------------------------------------------------------
Intercept              -4.444        0.942     -4.717     0.000     -6.290     -2.598
age_range_under_10      0.378        0.427      0.885     0.376     -0.459      1.214
age_range_18_24   

/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))
/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))


Results for outcome_suspect_arrested:
                Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: outcome_suspect_arrested
No. Observations: 2898    Method:             REML                    
No. Groups:       32      Scale:              722.4966                
Min. group size:  90      Log-Likelihood:     -13711.9036             
Max. group size:  92      Converged:          Yes                     
Mean group size:  90.6                                                
-----------------------------------------------------------------------
                       Coef.   Std.Err.     z     P>|z|  [0.025  0.975]
-----------------------------------------------------------------------
Intercept              30.263     3.180    9.518  0.000  24.031  36.494
age_range_under_10      1.157     1.560    0.742  0.458  -1.900   4.214
age_range_18_24         0.118     0.015    8.075  0.000   0.089   0.146
age_range_25_34        -0.119     0.030   -3.923  

/Users/cris/PycharmProjects/DC2-Group-15/.venv/lib/python3.9/site-packages/statsmodels/iolib/summary2.py:579: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dat = dat.applymap(lambda x: _formatter(x, float_format))
